## Imports

In [ ]:
import pdfplumber
import pandas as pd
import numpy as np
from datetime import datetime
import os
import re

# ==========================================

## Functions

In [ ]:
def get_date(str_date):
    date = datetime.strptime(str_date, '%d-%b%Y')
    return date

def is_date(date):
    try:
        get_date(date)
        return True
    except ValueError:
        return False

def is_float(str_float):
    try:
        float(str_float)
        if str_float[-3] == ".":
            return True
        else:
            return False
    except ValueError:
        return False

In [ ]:
def enter_row_into_statement_dict(date, details, amount, balance):
    global statement_dict
    statement_dict["Date"] += [date]
    statement_dict["Transaction Details"] += [details]
    statement_dict["Amount"] += [amount]
    statement_dict["Balance"] += [balance]
    
def clear_statement_dict():
    global statement_dict
    statement_dict["Date"] = []
    statement_dict["Transaction Details"] = []
    statement_dict["Amount"] = []
    statement_dict["Balance"] = []
    
def clear_statement_globals():
    global temp_row
    temp_row["date"] = ""
    temp_row["balance"] = ""
    temp_row["amount"] = ""
    temp_row["details"] = ""

In [ ]:
def adjust_date(line):
    date_regex = '[0-9]{2} [a-zA-Z]{3}'
    double_digit_date = re.search(date_regex, line[:6])
    
    if double_digit_date: 
        adjusted_line = line[:2] + '-' + line[3:]
    else:
        adjusted_line = line[:1] + '-' + line[2:]
    return adjusted_line

def test_for_date(line):
    date_regex = '([0-9]{2}|[0-9]) [a-zA-Z]{3}'
    date_in_line = re.search(date_regex, line[:6])

    if date_in_line:
        return True
    else:
        return False

def adjust_year(line, invoice):
    line_has_date = test_for_date(line)
    
    if line_has_date:
        line = adjust_date(line)

    invoice_is_jan = "-01." in invoice
    line_has_dec = "-Dec " in line
    year = invoice[invoice.find('/')+1:invoice.find('-')]

    if invoice_is_jan and line_has_dec:
        return line.replace("-Dec", f"-Dec{int(year) - 1}")
    elif not line_has_date:
        return line
    else:
        adjusted_date_with_year = f"{line.split()[0] + year} {' '.join(line.split()[1:])}"
        return adjusted_date_with_year

In [ ]:
def add_row_to_dic(items, starts_with_date, last_item_is_number):
    global temp_row

    has_two_amounts = is_float(items[-2])
    
    if starts_with_date:
        temp_row["date"] = get_date(items[0])

        if has_two_amounts:
            temp_row["balance"] = items[-1]
            temp_row["amount"] = items[-2]
            temp_row["details"] = " ".join(items[1:-2])
            enter_row_into_statement_dict(temp_row["date"], temp_row["details"], temp_row["amount"], temp_row["balance"])

        elif last_item_is_number:
            temp_row["amount"] = items[-1]
            temp_row["details"] = " ".join(items[1:-1])
            enter_row_into_statement_dict(temp_row["date"], temp_row["details"], temp_row["amount"], temp_row["balance"])

        else:
            temp_row["details"] = " ".join(items[1:])
            temp_row["amount"] = ""

    else:

        if has_two_amounts:
            temp_row["balance"] = items[-1]
            temp_row["amount"] = items[-2]
            temp_row["details"] = temp_row["details"] + " " + " ".join(items[0:-2])
            enter_row_into_statement_dict(temp_row["date"], temp_row["details"], temp_row["amount"], temp_row["balance"])
        
        elif last_item_is_number:            
            temp_row["amount"] = items[-1]

            if temp_row["unfinished-flag"] is True:
                temp_row["details"] = temp_row["details"] + " " + " ".join(items[0:-1])
                temp_row["unfinished-flag"] = False
            
            else:
                temp_row["details"] = " ".join(items[0:-1])

            enter_row_into_statement_dict(temp_row["date"], temp_row["details"], temp_row["amount"], temp_row["balance"])
        
        else:
            temp_row["amount"] = ""
            temp_row["details"] = " ".join(items)
            temp_row["unfinished-flag"] = True

def add_first_row_to_dic(items):
    global temp_row

    temp_row["date"] = get_date(items[0])
    temp_row["details"] = " ".join(items[1:-1])
    temp_row["balance"] = items[-1].replace(",","")
    enter_row_into_statement_dict(temp_row["date"], temp_row["details"], temp_row["amount"], temp_row["balance"])

def prepare_statement_dict(statement, invoice):
    for line in statement:
        items = adjust_year(line, invoice).split()

        last_item_is_number = is_float(items[-1])
        details_entered_yet = temp_row["details"] != ""
        starts_with_date = is_date(items[0])

        if starts_with_date and last_item_is_number and not details_entered_yet:
            add_first_row_to_dic(items)
        elif starts_with_date and last_item_is_number: 
            add_row_to_dic(items, starts_with_date=True, last_item_is_number=True)
        elif starts_with_date and not last_item_is_number: 
            add_row_to_dic(items, starts_with_date=True, last_item_is_number=False)
        elif not starts_with_date and last_item_is_number:
            add_row_to_dic(items, starts_with_date=False, last_item_is_number=True)
        elif not starts_with_date and not last_item_is_number:
            add_row_to_dic(items, starts_with_date=False, last_item_is_number=False)

In [ ]:
def remove_page_break(statement_raw):
    page_break = 'The Hongkong and Shanghai Banking Corporation'

    if page_break in " ".join(statement_raw):
        start = [i for i, s in enumerate(statement_raw) if page_break in s]
        end = [i for i, s in enumerate(statement_raw) if 'Date' in s]

        statement = statement_raw[0:start[0]] + statement_raw[end[0]+1:]
        return statement
    else:
        return statement_raw

def get_statement(text):
    for i, s in enumerate(text.split("\n")):
        if 'B/F BALANCE' in s:
            start = i
            break
    
    if 'HKD Current' in text:
        for i, s in reversed(list(enumerate(text.split("\n")))):
            if 'HKD Current' in s:
                end = i
                break
            
        statement_raw = [row.replace(",", "") for row in text.split("\n")[start:end]]

    else:
        for i, s in reversed(list(enumerate(text.split("\n")))):
            if 'Total Relationship Balance' in s:
                end = i
                break
        
        statement_raw = [row.replace(",", "") for row in text.split("\n")[start:end-1]]

    statement = remove_page_break(statement_raw)
    return statement

In [ ]:
def convert(invoices):
    global statement_dict

    for invoice in invoices:
        with pdfplumber.open(invoice) as pdf:
            text = " ".join([content.extract_text(x_tolerance=1) for content in pdf.pages])

        statement = get_statement(text)
        prepare_statement_dict(statement, invoice)
        clear_statement_globals()
        
    df = pd.DataFrame(statement_dict, columns= ["Date", "Transaction Details", "Amount", "Balance"])
    return df

# ==========================================

## Globals

In [ ]:
statement_dict = {
    "Date": [],
    "Transaction Details": [], 
    "Amount": [],
    "Balance": [] 
}

In [ ]:
temp_row = {
    "date": "",
    "balance": "",
    "amount": "",
    "details": "",
    "unfinished-flag": False
}

# ==========================================

## START HERE

In [ ]:
invoices = [f'mp_statements/{string}'for string in os.listdir(f'{os.getcwd()}/mp_statements') if string[0] != '.']
invoices

In [ ]:
df = convert(invoices).sort_values(by="Date")
df.head(20)

In [ ]:
df.to_csv("mp_statement.csv", index=False, mode='w+')

# ==========================================

## Clean Up

In [ ]:
clear_statement_dict()